In [ ]:
import os
import yaml
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.graph_objects as go
from epynet import Network
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from stable_baselines import DQN
from wdsEnv import wds

In [ ]:
def assemble_junc_coordinates(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)
    return junc_x, junc_y, junc_z

In [ ]:
def assemble_pipe_coords(wds):
    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(float('nan'))

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(float('nan'))

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(float('nan'))
    return pipe_x, pipe_y, pipe_z

In [ ]:
def build_2d_traces(junc_coords, pipe_coords):
    junc_trace = go.Scatter(
        x = junc_coords[0],
        y = junc_coords[1],
        mode='markers'
        )
    
    pipe_trace = go.Scatter(
        x = pipe_coords[0],
        y = pipe_coords[1],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [ ]:
def demand_plot(junc_trace, pipe_trace, wds):
    junc_trace.marker = dict(
    showscale = True,
    colorscale = 'YlGnBu',
    reversescale = True,
    color = list(wds.junctions.basedemand),
    size = 10,
    colorbar = dict(
        thickness = 15,
        title = 'Water consumption, m**3/h',
        xanchor = 'left',
        titleside = 'right'
        )
    )

    fig = go.Figure(
    data = [pipe_trace, junc_trace],
    layout = go.Layout(
        #title = wds_btn.value,
        titlefont_size = 16,
        showlegend = False,
        hovermode = 'closest',
        margin = dict(b=20,l=5,r=5,t=40),
        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False)
        )
    )
    return fig

# Main code

## Plot

In [ ]:
wds_fname = 'anytown_master.inp'
wds_topo = Network(os.path.join('water_networks', wds_fname))
orig_demands = wds_topo.junctions.basedemand
wds_topo.solve()

In [ ]:
junc_coords = assemble_junc_coordinates(wds_topo)
pipe_coords = assemble_pipe_coords(wds_topo)
junc_trace, pipe_trace = build_2d_traces(junc_coords, pipe_coords)
fig = demand_plot(junc_trace, pipe_trace, wds_topo)
fig.show()

## RL

In [ ]:
hyperparams_fn  = 'anytownMaster'
model_fn        = 'anytownHO1-best'

pathToParams= os.path.join('experiments', 'hyperparameters', hyperparams_fn+'.yaml')
with open(pathToParams, 'r') as fin:
    hparams = yaml.load(fin, Loader=yaml.Loader)
pathToModel = os.path.join('experiments', 'models', model_fn+'.zip')

env = wds(
        wds_name        = hparams['env']['waterNet']+'_master',
        speed_increment = hparams['env']['speedIncrement'],
        episode_len     = hparams['env']['episodeLen'],
        pump_groups     = hparams['env']['pumpGroups'],
        total_demand_lo = hparams['env']['totalDemandLo'],
        total_demand_hi = hparams['env']['totalDemandHi'],
        reset_orig_pump_speeds  = hparams['env']['resetOrigPumpSpeeds'],
        reset_orig_demands      = hparams['env']['resetOrigDemands']
)

model   = DQN.load(pathToModel)

obs = env.reset()
while not env.done:
    act, _              = model.predict(obs, deterministic=True)
    obs, reward, _, _   = env.step(act, training=False)
    print(reward)